In [1]:
import cv2
import tensorflow as tf
import numpy as np

In [2]:
model = tf.keras.models.load_model("CNNWHITE.model")

In [3]:
def prepare(ima):
    IMG_SIZE = 100  # 50 in txt-based
    img_array = cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    img_array=img_array/255.0  # filepathread in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1,IMG_SIZE, IMG_SIZE,1)

In [4]:
def paa_skin(frame):    
    ycrcb=cv2.cvtColor(frame,cv2.COLOR_BGR2YCrCb)
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    b,g,r = cv2.split (frame)
    Y,Cr,Cb=cv2.split(ycrcb)
    H,S,V=cv2.split(hsv)
    #S=S/(H+S+V)
    
    result=((0.0 <=H)*(H <= 50.0)*(0.23 <= S)*(S<= 0.68)*(r > 95)*(g > 40)*(b > 20)*(r > g)*(r > b)*(abs(r - g)> 15)+(r > 95)*(g > 40)*(b > 20)*(r > g)*(r > b)*(abs(r - g ) > 15)*(Cr > 135)*(Cb > 85)*(Y > 80)*(Cr <= (1.5862*Cb)+20)*(Cr>=(0.3448*Cb)+76.2069)*(Cr >= (-4.5652*Cb)+234.5652)*(Cr <= (-1.15*Cb)+301.75)*(Cr <= (-2.2857*Cb)+432.85))*255
    result=cv2.merge((result,result,result))
    img2=frame&result
    img3=255-result
    img4=img2+img3
    cv2.imwrite('image.jpeg',img4)
    img1=cv2.imread('image.jpeg')
    
    return img1

In [6]:
cap=cv2.VideoCapture(0)
cx=100
cy=100
rw=300
rh=300
while True:
    ret, image=cap.read()
    new=paa_skin(image)
    cv2.imshow('new',new)
    pre=new[cx:rw,cy:rh]
    prediction = model.predict([prepare(pre)])
    prediction=np.argmax(prediction)
    x1=prediction
    try:
         cv2.putText(new,str(x1),(50,50),cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,0,0),lineType=cv2.LINE_AA)
    except:
          print("Variable x1 is not empty")
    cv2.rectangle(new,(cx,cy),(cx+rw,cy+rh),(0,255,0),2)
    cv2.imshow('new',new)
    keypress = cv2.waitKey(1) & 0xFF
    if keypress == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()